In [15]:
import molli as ml
import pyvista as pv
from PIL import ImageColor
from matplotlib.colors import ListedColormap
import numpy as np

pv.set_jupyter_backend("pythreejs")

with ml.files.mol2.pdb_4a05.open() as f:
    _mol = ml.chem.Molecule.from_mol2(f)

# mol = _mol.heavy
mol = _mol

for a in mol.atoms:
    if a.element.symbol == "N":
        a.dummy = True

mol.translate(-1 * mol.centroid())
a_sizes = [a.cov_radius_1 for a in mol.atoms]

# sph = pv.Sphere(theta_resolution=24, phi_resolution=24)

plotter = pv.Plotter(multi_samples=4)

atoms = pv.MultiBlock()
for i, a in enumerate(mol.atoms):
    a_size = a.cov_radius_1 * 0.4
    if a.dummy:
        atoms.append(pv.Cube(mol.coords[i], a_size, a_size, a_size))
    else:
        atoms.append(
            pv.Sphere(
                a_size / 2, center=mol.coords[i], phi_resolution=12, theta_resolution=12
            )
        )

bonds = pv.MultiBlock()
for j, b in enumerate(mol.bonds):
    r1, r2 = mol.bond_coords(b)
    bonds.append(pv.Tube(r1, r2, radius=0.03, n_sides=16))


actor, mapper = plotter.add_composite(
    atoms,
    smooth_shading=True,
    culling=True,
    diffuse=0.6,
    specular=0.2,
)

for i, a in enumerate(mol.atoms):
    mapper.block_attr[i + 1].color = a.color_cpk

actor, mapper = plotter.add_composite(
    bonds,
    color=(0.9, 0.9, 0.9),
    smooth_shading=True,
    culling=True,
    diffuse=0.7,
    specular=0.3,
)

plotter.enable_anti_aliasing(aa_type="fxaa", multi_samples=2)
plotter.background_color = "black"
plotter.view_xy()
plotter.add_axes()


C:\SAS\programs\repo\molli\molli\chem\atom.py:367: UserWarning: Replaced illegal characters in molecule name: ***** --> _____
  warn(f"Replaced illegal characters in molecule name: {_value} --> {sub}")


<vtkmodules.vtkRenderingAnnotation.vtkAxesActor(0x000002F7F597F7A0) at 0x000002F78047E260>

In [16]:
# plotter.show(jupyter_backend="ipygany", window_size=(1000,600))
plotter.show(window_size=(1000,600))

KeyboardInterrupt: 